<a href="https://colab.research.google.com/github/HappyJJins/Jeju_CreditCard/blob/master/Error0727.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive

ROOT = '/content/drive'
drive.mount(ROOT)

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from itertools import product

# 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 통계
from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [4]:
class Model:
    # Initialization method of the Model class about definition of the data
    # Input : split_data(Training/Test set of the independent value and the dependent value)
    #       : num(encoding_data)
    # Definition of the X
    def __init__(self, split_data, num):
        self.X_train = split_data[0]
        self.X_test = split_data[1]
        self.y_train = split_data[2]
        self.y_test = split_data[3]
        self.encoding_data = num

    # the rmsle between the actual value and predicted value
    def rmsle(self, y, pred): 
        log_y = np.log1p(y)
        log_pred = np.log1p(pred)
        squared_error = (log_y - log_pred)**2
        rmsle = np.sqrt(np.mean(squared_error))
        print('Test Data RMSLE: {0:.3f}'.format(rmsle))
        return round(rmsle, 3)

    # get best estimator and rmsle through GridSearchCV
    def best_params_model(self, model, params):
        cv_model = GridSearchCV(model, param_grid=params, scoring="neg_mean_squared_error", cv = 5)
        cv_model.fit(self.X_train, self.y_train)
        print("----", model.__class__.__name__, "----")
        print("GridSearchCV 최적 하이퍼 파라미터 :", cv_model.best_params_)

        eval_pred = cv_model.predict(self.X_test)
        eval_pred = np.expm1(eval_pred)
        rmsle_ = self.rmsle(self.y_test, eval_pred)

        return  cv_model.best_estimator_, rmsle_

    # Process about finding the ratio of XGB and GBM and LGB
    def get_model(self):
        xgb = XGBRegressor(random_state=0)
        gbm = GradientBoostingRegressor(random_state=0)
        lgb = LGBMRegressor(random_state=0)

        params = {'n_estimators': [1000, 2000]}

        models = [xgb, gbm, lgb]
        best_models = []
        rmsles = []
        
        for model in models:
            new_model = self.best_params_model(model, params)[0]
            new_rmsle = self.best_params_model(model, params)[1]
            best_models.append(new_model)
            rmsles.append(new_rmsle)

        self.xgb_reg = best_models[0]
        self.gbm_reg = best_models[1]
        self.lgb_reg = best_models[2]

        self.xgb_rmsle = rmsles[0]
        self.gbm_rmsle = rmsles[1]
        self.lgb_rmsle = rmsles[2]

        xgb_pred = self.xgb_reg.predict(self.X_test)
        xgb_pred = np.expm1(xgb_pred)

        gbm_pred = self.gbm_reg.predict(self.X_test)
        gbm_pred = np.expm1(gbm_pred)

        lgb_pred = self.lgb_reg.predict(self.X_test)
        lgb_pred = np.expm1(lgb_pred)

        pred = np.array([xgb_pred, gbm_pred, lgb_pred])
        pred = np.transpose(pred)

        rmsle_sum = self.xgb_rmsle + self.gbm_rmsle + self.lgb_rmsle

        self.xgb_per = self.xgb_rmsle / rmsle_sum
        self.gbm_per = self.gbm_rmsle / rmsle_sum
        self.lgb_per = self.lgb_rmsle / rmsle_sum
        
        final = xgb_pred*self.xgb_per + gbm_pred*self.gbm_per + lgb_pred*self.lgb_per
        print("--final RSMLE--")
        self.rmsle(self.y_test, final)

    # making the prediction template about 04/07/2020
    def make_temp(self):
        CARD_SIDO_NMs = self.encoding_data['CARD_SIDO_NM'].unique()
        STD_CLSS_NMs  = self.encoding_data['STD_CLSS_NM'].unique()
        HOM_SIDO_NMs  = self.encoding_data['HOM_SIDO_NM'].unique()
        AGEs          = self.encoding_data['AGE'].unique()
        SEX_CTGO_CDs  = self.encoding_data['SEX_CTGO_CD'].unique()
        FLCs          = self.encoding_data['FLC'].unique()
        years         = [2020]
        months        = [4, 7]

        comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs,HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, years, months]
        temp = np.array(list(product(*comb_list)))

        train_features = self.encoding_data.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
        tmp = pd.DataFrame(data=temp, columns=train_features.columns)

        return tmp

    # making submission
    def make_sub(self, temp):
        xgb_pred = self.xgb_reg.predict(temp)
        xgb_pred = np.expm1(xgb_pred)

        gbm_pred = self.gbm_reg.predict(temp)
        gbm_pred = np.expm1(gbm_pred)

        lgb_pred = self.lgb_reg.predict(temp)
        lgb_pred = np.expm1(lgb_pred)

        final_rmsle = xgb_pred*self.xgb_per + gbm_pred*self.gbm_per + lgb_pred*self.lgb_per

        temp['AMT'] = np.round(final_rmsle, 0)
        temp['REG_YYMM'] = temp['year']*100 + temp['month']
        temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
        temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop = False)

        temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
        temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

        return temp

In [ ]:
def sampling(sido):
    df = train[train['CARD_SIDO_NM']==sido]
    df = df.sample(n=10000)
    return df

In [ ]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [ ]:
def data_pre(data):
    data = data.fillna('')
    data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
    data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
    data = data.drop(['REG_YYMM'], axis = 1)

    return data

In [ ]:
def encoding(data):

    global encoders
    dtypes = data.dtypes
    encoders = {}
    for column in data.columns:
        if str(dtypes[column]) == 'object':
            encoder = LabelEncoder()
            encoder.fit(data[column])
            encoders[column] = encoder

    data_num = data.copy()

    for column in encoders.keys():
        encoder = encoders[column]
        data_num[column] = encoder.transform(data[column])

    return data_num

In [ ]:
def split(data_num):
    split_data = []

    X_data, y_data = data_num.loc[:, data_num.columns != 'AMT'], data_num['AMT']
    X_data = X_data.drop(['CSTMR_CNT', 'CNT'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=126, shuffle=True)
    y_train = np.log1p(y_train)
    
    return X_train, X_test, y_train, y_test

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Jeju/data/jeju_data_ver1/201901-202003.csv')
sido_list = train['CARD_SIDO_NM'].unique().tolist()

In [ ]:
for sido in sido_list:
  temp = sampling(sido).reset_index(drop=True)

  temp = data_pre(temp)
  temp = temp.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

  temp_num = encoding(temp)
  temp_data = split(temp_num)

  temp = Model(temp_data, temp_num)

  temp.get_model()

  temp_temp = temp.make_temp()

  temp_sub = temp.make_sub(temp_temp)

  globals()['sub_{}'.format(sido)] = temp_sub

[13:33:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
from numpy import inf
def sido_diff(sido):
  df = train[train['CARD_SIDO_NM'] == sido]
  df_19 = df[df['REG_YYMM'] == 201903]
  df_20 = df[df['REG_YYMM'] == 202003]
  diff = pd.concat([df_19.groupby('STD_CLSS_NM').AMT.sum(), df_20.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys = ['1903_AMT', '2003_AMT']).reset_index()
  diff = diff.fillna(0)
  diff['diff'] = (diff['1903_AMT']-diff['2003_AMT'])/diff['1903_AMT']
  diff['diff'] = diff['diff'].apply(lambda x: 0 if x == -inf else x) # 그 이유는 뒤에
  diff.columns = ['STD_CLSS_NM', '1903_AMT', '2003_AMT', 'diff']
  diff = diff[['STD_CLSS_NM', 'diff']]
  diff = diff.sort_values(by='diff', ascending=False).reset_index(drop=True)

  return diff

In [ ]:
# sido_diff에서 주석 달린 행의 코드를 넣지 않을 경우
sido_diff('제주')

,STD_CLSS_NM,diff
0,그외 기타 스포츠시설 운영업,1.000000
1,정기 항공 운송업,0.940082
2,휴양콘도 운영업,0.931019
3,여행사업,0.845879
4,관광 민예품 및 선물용품 소매업,0.836989
5,전시 및 행사 대행업,0.829149
6,면세점,0.658461
7,내항 여객 운송업,0.617504
8,마사지업,0.579779
9,호텔업,0.561504


In [ ]:
sido = '제주'
df = train[train['CARD_SIDO_NM'] == sido]
df_19 = df[df['REG_YYMM'] == 201903]
df_20 = df[df['REG_YYMM'] == 202003]
diff = pd.concat([df_19.groupby('STD_CLSS_NM').AMT.sum(), df_20.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys = ['1903_AMT', '2003_AMT']).reset_index()
diff = diff.fillna(0)
diff['diff'] = (diff['1903_AMT']-diff['2003_AMT'])/diff['1903_AMT']
diff.columns = ['STD_CLSS_NM', '1903_AMT', '2003_AMT', 'diff']
diff = diff.sort_values(by='diff', ascending=False).reset_index(drop=True)
diff

,STD_CLSS_NM,1903_AMT,2003_AMT,diff
0,그외 기타 스포츠시설 운영업,1.051300e+07,0.000000e+00,1.000000
1,정기 항공 운송업,3.333231e+09,1.997219e+08,0.940082
2,휴양콘도 운영업,5.613140e+08,3.871976e+07,0.931019
3,여행사업,2.552282e+07,3.933600e+06,0.845879
4,관광 민예품 및 선물용품 소매업,3.173647e+08,5.173391e+07,0.836989
5,전시 및 행사 대행업,3.236805e+08,5.530123e+07,0.829149
6,면세점,7.585381e+09,2.590700e+09,0.658461
7,내항 여객 운송업,1.736811e+08,6.643240e+07,0.617504
8,마사지업,6.330000e+06,2.660000e+06,0.579779
9,호텔업,1.128726e+09,4.949414e+08,0.561504


201903 AMT 값이 0이면 증감율 계산시 분모가 0이 되므로 결괏값이 -inf가 됨


이 완성코드를 구현하기 전,
생성된 증감율 데이터 프레임을 이용, 기존에 생성해 둔 지역별 예측 값에 증감율을 반영하여 제출하려 했으나, 만든 submission 파일과 정답지의 데이터 형식이 맞지 않아 제출이 되지 않는 현상이 발생.

확인 결과, 만든 submission의 AMT 컬럼이 float64, 정답지의 AMT 컬럼은 int로, 데이터 타입이 맞지 않음을 확인, int로 변환하려 했으나 두 데이터 프레임에서 문제가 생김.

1. diff_제주 : 2019년 3월 AMT 중 값이 0인 업종이 존재해, 증감율이 -inf가 나오게 됨 → diff 병합 전에 이상치 처리
2. diff_울산 : diff 데이터 프레임 생성 중엔 문제가 없으나, 병합 후 예측치에 존재하는 업종 중 존재하지 않는 업종이 존재, nan값 발생 → 병합 후 결측치 처리

In [ ]:
# 각 시도별 증감율 데이터프레임 생성
for sido in sido_list:
  tmp = sido_diff(sido)

  globals()['diff_{}'.format(sido)] = tmp

In [ ]:
diff_제주

,STD_CLSS_NM,diff
0,그외 기타 스포츠시설 운영업,1.000000
1,정기 항공 운송업,0.940082
2,휴양콘도 운영업,0.931019
3,여행사업,0.845879
4,관광 민예품 및 선물용품 소매업,0.836989
5,전시 및 행사 대행업,0.829149
6,면세점,0.658461
7,내항 여객 운송업,0.617504
8,마사지업,0.579779
9,호텔업,0.561504


In [ ]:
# 시도별로 뽑은 샘플로 예측한 df와 전체 데이터에서 2019년 03월과 2020년 03월을 뽑아 시도별로 증감율 계산한 df -> 하나로 병합
test_subs = [sub_강원, sub_경기, sub_경남, sub_경북, sub_광주, sub_대구, sub_대전, sub_부산, sub_서울, sub_세종, sub_울산, sub_인천, sub_전남, sub_전북, sub_제주, sub_충남, sub_충북]
test_diffs = [diff_강원, diff_경기, diff_경남, diff_경북, diff_광주, diff_대구, diff_대전, diff_부산, diff_서울, diff_세종, diff_울산, diff_인천, diff_전남, diff_전북, diff_제주, diff_충남, diff_충북]

In [ ]:
# 시도별로 뽑은 샘플로 예측한 df json 파일로 저장
for i in range(len(sido_list)):
  test_subs[i].to_json('/content/drive/My Drive/Colab Notebooks/Jeju/data/1903to2003_local/'+sido_list[i]+'.json', orient='table')

In [ ]:
# 혹시 몰라서 복사본 만듬
subs = test_subs.copy()
diffs = test_diffs.copy()

In [ ]:
# 샘플로 예측한 값에서 증감율 고려해 수정
for i in range(len(sido_list)):
  subs[i] = subs[i].merge(diffs[i][['STD_CLSS_NM','diff']], how='left', on='STD_CLSS_NM')
  subs[i]['AMT'] = subs[i]['AMT']*(1-subs[i]['diff'])
  subs[i] = subs[i][['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [ ]:
# 모든 시도 데이터 합치기
test = pd.concat(subs)
test = test.sort_values(by = ['REG_YYMM', 'CARD_SIDO_NM']).reset_index(drop = True)

In [ ]:
test

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,2.271906e+08
1,202004,강원,골프장 운영업,4.116734e+08
2,202004,강원,과실 및 채소 소매업,7.117371e+07
3,202004,강원,관광 민예품 및 선물용품 소매업,2.322025e+07
4,202004,강원,그외 기타 스포츠시설 운영업,1.424509e+07
...,...,...,...,...
1207,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.387591e+07
1208,202007,충북,한식 음식점업,8.432970e+07
1209,202007,충북,호텔업,2.166644e+07
1210,202007,충북,화장품 및 방향제 소매업,5.491836e+07


In [ ]:
# nan값 확인
test[test['AMT'].isnull()]

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
454,202004,전남,택시 운송업,NaN
583,202004,충북,버스 운송업,NaN
1060,202007,전남,택시 운송업,NaN
1189,202007,충북,버스 운송업,NaN


위의 NaN값에 해당하는 '전남' - '택시/버스 운송업' 과 '충북' - '택시/버스 운송업' 데이터는 전체 파일에는 있으나 샘플링 시 추출하지 못한 데이터라고 할 수 있다.
이 결측치를 어떻게 처리해 줄 것인가.


결측치를 가지는 지역과 업종을 전체 데이터에서 확인

In [ ]:
train[(train['CARD_SIDO_NM']=='전남') & (train['STD_CLSS_NM']=='택시 운송업')]

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
7996298,201905,전남,곡성군,택시 운송업,광주,남구,20s,2,1,3,30350,5


In [ ]:
train[(train['CARD_SIDO_NM']=='전남') & (train['STD_CLSS_NM']=='버스 운송업')]

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
1359196,201901,전남,광양시,버스 운송업,전남,광양시,10s,1,1,3,24800,8
1359197,201901,전남,광양시,버스 운송업,전남,광양시,20s,1,1,36,405980,81
1359198,201901,전남,광양시,버스 운송업,전남,광양시,20s,1,2,7,50100,12
1359199,201901,전남,광양시,버스 운송업,전남,광양시,20s,2,1,9,91800,24
1359200,201901,전남,광양시,버스 운송업,전남,광양시,30s,1,1,3,13900,4
...,...,...,...,...,...,...,...,...,...,...,...,...
24497665,202003,전남,광양시,버스 운송업,전남,광양시,40s,2,3,13,165800,18
24497666,202003,전남,광양시,버스 운송업,전남,광양시,50s,1,4,44,849150,104
24497667,202003,전남,광양시,버스 운송업,전남,광양시,50s,2,4,10,128150,16
24497668,202003,전남,광양시,버스 운송업,전남,광양시,60s,1,5,10,156110,24


In [ ]:
train[(train['CARD_SIDO_NM']=='충북') & (train['STD_CLSS_NM']=='택시 운송업')]

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
1598594,201901,충북,청주시 흥덕구,택시 운송업,충북,청주시 서원구,30s,2,2,4,23200,4
1598595,201901,충북,청주시 흥덕구,택시 운송업,충북,청주시 흥덕구,20s,1,1,4,21000,4
1598596,201901,충북,청주시 흥덕구,택시 운송업,충북,청주시 흥덕구,20s,2,1,5,23600,5
1598597,201901,충북,청주시 흥덕구,택시 운송업,충북,청주시 흥덕구,50s,1,4,3,15500,3
1598598,201901,충북,청주시 흥덕구,택시 운송업,충북,청주시 흥덕구,50s,2,4,3,12700,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24691514,202003,충북,청주시 흥덕구,택시 운송업,충북,청주시 청원구,40s,1,3,3,27820,4
24691515,202003,충북,청주시 흥덕구,택시 운송업,충북,청주시 청원구,40s,2,3,3,12460,3
24691516,202003,충북,청주시 흥덕구,택시 운송업,충북,청주시 청원구,50s,2,4,3,16340,3
24691517,202003,충북,청주시 흥덕구,택시 운송업,충북,청주시 흥덕구,20s,1,1,4,23200,4


In [ ]:
train[(train['CARD_SIDO_NM']=='충북') & (train['STD_CLSS_NM']=='버스 운송업')]

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
1594664,201901,충북,청주시 흥덕구,버스 운송업,충북,청주시 서원구,50s,2,4,3,28000,3
6511619,201904,충북,청주시 흥덕구,버스 운송업,충북,청주시 청원구,30s,1,2,3,26000,3
6511620,201904,충북,청주시 흥덕구,버스 운송업,충북,청주시 흥덕구,60s,2,5,3,18000,3
8283146,201905,충북,청주시 흥덕구,버스 운송업,충북,청주시 서원구,60s,1,5,4,32000,5
9996491,201906,충북,청주시 흥덕구,버스 운송업,경기,성남시 분당구,40s,1,3,3,32000,4
9996492,201906,충북,청주시 흥덕구,버스 운송업,세종,NaN,40s,1,3,3,183000,12
11718208,201907,충북,청주시 흥덕구,버스 운송업,충북,청주시 서원구,50s,1,4,3,21000,3
11718209,201907,충북,청주시 흥덕구,버스 운송업,충북,청주시 흥덕구,40s,1,3,3,30000,3
13597044,201908,충북,청주시 흥덕구,버스 운송업,충북,청주시 흥덕구,30s,1,2,3,20000,3
15331868,201909,충북,청주시 흥덕구,버스 운송업,경기,성남시 분당구,50s,1,4,3,41000,3


In [ ]:
nan = test[test['AMT'].isnull()]
train_03 = train[train['REG_YYMM']==202003]

In [ ]:
for i in range(len(nan)):
  print(sum(train_03[(train_03['CARD_SIDO_NM']==nan.iloc[i]['CARD_SIDO_NM']) & (train_03['STD_CLSS_NM']==nan.iloc[i]['STD_CLSS_NM'])]['AMT']))

0
0
0
0


위에서 결측치를 가지는 시도와 업종에 해당하는 2020년 03월의 AMT는 모두 0이었다.
따라서 결측치를 0으로 대체해도 무방하다고 판단.

In [ ]:
# 결측치 → 0으로
test['AMT'] = test['AMT'].fillna(0)

In [ ]:
# test['AMT']의 데이터 타입을 int로 변환
test['AMT'] = test['AMT'].apply(int)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   REG_YYMM      1212 non-null   int64 
 1   CARD_SIDO_NM  1212 non-null   object
 2   STD_CLSS_NM   1212 non-null   object
 3   AMT           1212 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 38.0+ KB


In [ ]:
project_id = 'jeju-282707'
client = bigquery.Client(project = 'jeju-282707')
submission = client.query('''
  SELECT 
    * 
  FROM `jeju-282707.jeju_data_ver1.submission` 
  ''').to_dataframe()

In [ ]:
# 제출파일 양식에 맞게 가공
final = submission.drop(['AMT'], axis=1)
final = final.merge(test, on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
final

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,227190609.0
1,1,202004,강원,골프장 운영업,411673404.0
2,2,202004,강원,과실 및 채소 소매업,71173708.0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,23220254.0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,23875910.0
1390,1390,202007,충북,한식 음식점업,84329698.0
1391,1391,202007,충북,호텔업,21666439.0
1392,1392,202007,충북,화장품 및 방향제 소매업,54918363.0


In [ ]:
# 결측치 확인 -> 샘플링 시 추출하지 못한 시도별 업종 데이터라 할 수 있다.
final[final['AMT'].isnull()]

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
14,14,202004,강원,면세점,NaN
28,28,202004,강원,자동차 임대업,NaN
30,30,202004,강원,정기 항공 운송업,NaN
35,35,202004,강원,택시 운송업,NaN
...,...,...,...,...,...
1347,1347,202007,충남,택시 운송업,NaN
1357,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,NaN
1376,1376,202007,충북,여행사업,NaN
1381,1381,202007,충북,자동차 임대업,NaN


In [ ]:
final_nan = final[final['AMT'].isnull()]
for i in range(len(final_nan)):
  print(sum(train_03[(train_03['CARD_SIDO_NM']==final_nan.iloc[i]['CARD_SIDO_NM']) & (train_03['STD_CLSS_NM']==final_nan.iloc[i]['STD_CLSS_NM'])]['AMT']))

결측치에 해당하는 시도별 업종 소비금액을 실제 2020년 03월에 적용시켰을 때 꽤 큰 값들이 종종 보임.  
결측치를 2020년 03월 데이터를 그대로 쓸 것

In [ ]:
# 제출 df에서 결측치 값을 가지는 행들을 추출해 final_nan으로 저장
final_nan = final_nan.reset_index(drop=True)
final_nan

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
1,14,202004,강원,면세점,NaN
2,28,202004,강원,자동차 임대업,NaN
3,30,202004,강원,정기 항공 운송업,NaN
4,35,202004,강원,택시 운송업,NaN
...,...,...,...,...,...
177,1347,202007,충남,택시 운송업,NaN
178,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,NaN
179,1376,202007,충북,여행사업,NaN
180,1381,202007,충북,자동차 임대업,NaN


In [ ]:
final_nan['AMT'] = final_nan['AMT'].fillna(0)
final_nan['AMT'] = final_nan['AMT'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
final_nan

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0
14,14,202004,강원,면세점,0
28,28,202004,강원,자동차 임대업,0
30,30,202004,강원,정기 항공 운송업,0
35,35,202004,강원,택시 운송업,0
...,...,...,...,...,...
1347,1347,202007,충남,택시 운송업,0
1357,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,0
1376,1376,202007,충북,여행사업,0
1381,1381,202007,충북,자동차 임대업,0


### 여기서 계속 에러뜸 !!!!!!

In [ ]:
for idx, x in final_nan.iterrows():
  x['AMT'] = sum(train_03[(train_03['CARD_SIDO_NM']==x['CARD_SIDO_NM']) & (train_03['STD_CLSS_NM']==x['STD_CLSS_NM'])]['AMT'])

In [ ]:
final_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 4 to 1383
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            182 non-null    int64 
 1   REG_YYMM      182 non-null    int64 
 2   CARD_SIDO_NM  182 non-null    object
 3   STD_CLSS_NM   182 non-null    object
 4   AMT           182 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 8.5+ KB


In [ ]:
train_03.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1260508 entries, 23437284 to 24697791
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   REG_YYMM      1260508 non-null  int64 
 1   CARD_SIDO_NM  1260508 non-null  object
 2   CARD_CCG_NM   1255817 non-null  object
 3   STD_CLSS_NM   1260508 non-null  object
 4   HOM_SIDO_NM   1260508 non-null  object
 5   HOM_CCG_NM    1253658 non-null  object
 6   AGE           1260508 non-null  object
 7   SEX_CTGO_CD   1260508 non-null  int64 
 8   FLC           1260508 non-null  int64 
 9   CSTMR_CNT     1260508 non-null  int64 
 10  AMT           1260508 non-null  int64 
 11  CNT           1260508 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 125.0+ MB


In [ ]:
final_nan

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0
14,14,202004,강원,면세점,0
28,28,202004,강원,자동차 임대업,0
30,30,202004,강원,정기 항공 운송업,0
35,35,202004,강원,택시 운송업,0
...,...,...,...,...,...
1347,1347,202007,충남,택시 운송업,0
1357,1357,202007,충북,그외 기타 분류안된 오락관련 서비스업,0
1376,1376,202007,충북,여행사업,0
1381,1381,202007,충북,자동차 임대업,0


#### 에러를 잡기 위해 몇시간을 투자한 결과 얻은 것

In [ ]:
a = pd.DataFrame([[0.0, 2], [3.0, 4]])
for idx, x in a.iterrows():
  x[0] = 10

In [ ]:
a

,0,1
0,0.0,2
1,3.0,4


와.......  
type 다르면 안 바뀜

In [ ]:
# 제출파일 생성
final.to_csv('/content/drive/My Drive/Colab Notebooks/Jeju/submission.csv', encoding='utf-8-sig', index=False)